<h2 style="text-align:center;font-size:200%;;">Industrial  Machine Anomaly Detection</h2>
<h3  style="text-align:center;">Keywords : <span class="label label-success">IoT</span> <span class="label label-success">Anomaly Detection</span> <span class="label label-success">Model Interpretability</span> <span class="label label-success">Model Comparison</span></h3>

# Table of Contents<a id='top'></a>
>1. [Overview](#1.-Overview)  
>    * [Project Detail](#Project-Detail)
>    * [Goal of this notebook](#Goal-of-this-notebook)
>1. [Import libraries](#2.-Import-libraries)
>1. [Load the dataset](#3.-Load-the-dataset)
>1. [Pre-processing](#4.-Pre-processing)
>    * [Anomaly Points](#Anomaly-Points)
>    * [Datetime Information](#Datetime-Information)
>1. [EDA](#5.-EDA)  
>    * [Basic Analysis](#Basic-Analysis)
>    * [Time Series Analysis](#Time-Series-Analysis)
>1. [Modeling](#6.-Modeling)
>    * [Model1. Hotelling's T2](#Model1.-Hotelling's-T2)
>    * [Model2. One-Class SVM](#Model2.-One\-Class-SVM)
>    * [Model3. Isolation Forest](#Model3.-Isolation-Forest)
>    * [Model4. LOF](#Model4.-LOF)
>    * [Model5. ChangeFinder](#Model5.-ChangeFinder)
>    * [Model6. Variance Based Method](#Model6.-Variance-Based-Method)
>    * [Model Comparison](#Model-Comparison)
>1. [Conclusion](#7.-Conclusion)
>1. [References](#8.-References)

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 1. Overview
## Project Detail
>In this project, we use [NAB-dataset](https://www.kaggle.com/boltzmannbrain/nab), which is a novel benchmark for evaluating algorithms for anomaly detection in several fields.  
>There are 58 timeseries data from various kind of sources.
>* **Real data**
>    * realAWSCloudwatch
>    * realAdExchange
>    * realKnownCause
>    * realTraffic
>    * realTweets
>* **Artificial data**
>    * artificialNoAnomaly
>    * artificialWithAnomaly
>
>In these dataset above, I picked up and analyzed **'machine_temperature_system_failure'** from realKnownCause dataset based on my buissiness interests.  
>This dataset does not include acutual anomaly point, so we need to refer to the [NAB github page](https://github.com/numenta/NAB/blob/master/labels/combined_windows.json).

## Goal of this notebook
>* Practice data pre-processing technique
>* Practice EDA technique to deal with time-series data
>* Practice visualising technique
>* Practice anomaly detection modeling technique
>    * from simple techniques to complex techniques
>* Practice improving model interpretability technique
>    * SHAP

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 2. Import libraries

In [ ]:
import numpy as np
import pandas as pd
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')
from matplotlib import pyplot as plt
import seaborn as sns
import os
import scipy.stats as stats
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
!pip install changefinder
import changefinder
from sklearn.metrics import f1_score
import shap
shap.initjs()
from tabulate import tabulate
from IPython.display import HTML, display

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 3. Load the dataset
>As above, we use 'machine_temperature_system_failure.csv' for our analysis.  
>According to dataset information, it has the following features : 
>* Temperature sensor data of an internal component of a large, industrial mahcine.
>* The first anomaly is a planned shutdown of the machine. 
>* The second anomaly is difficult to detect and directly led to the third anomaly, a catastrophic failure of the machine.

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input/nab/realKnownCause/realKnownCause'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/nab/realKnownCause/realKnownCause/machine_temperature_system_failure.csv",low_memory=False)
print(f'machine_temperature_system_failure.csv : {df.shape}')
df.head(3)

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 4. Pre-processing

## Anomaly Points
>We can get anomaly points information [here](https://github.com/numenta/NAB/blob/master/labels/combined_windows.json)

In [ ]:
anomaly_points = [
        ["2013-12-10 06:25:00.000000","2013-12-12 05:35:00.000000"],
        ["2013-12-15 17:50:00.000000","2013-12-17 17:00:00.000000"],
        ["2014-01-27 14:20:00.000000","2014-01-29 13:30:00.000000"],
        ["2014-02-07 14:55:00.000000","2014-02-09 14:05:00.000000"]
]

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
#is anomaly? : True => 1, False => 0
df['anomaly'] = 0
for start, end in anomaly_points:
    df.loc[((df['timestamp'] >= start) & (df['timestamp'] <= end)), 'anomaly'] = 1

## Datetime Information

In [ ]:
df['year'] = df['timestamp'].apply(lambda x : x.year)
df['month'] = df['timestamp'].apply(lambda x : x.month)
df['day'] = df['timestamp'].apply(lambda x : x.day)
df['hour'] = df['timestamp'].apply(lambda x : x.hour)
df['minute'] = df['timestamp'].apply(lambda x : x.minute)

In [ ]:
df.index = df['timestamp']
df.drop(['timestamp'], axis=1, inplace=True)
df.head(3)

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 5. EDA

## Basic Analysis

In [ ]:
count = hv.Bars(df.groupby(['year','month'])['value'].count()).opts(ylabel="Count", title='Year/Month Count')
mean = hv.Bars(df.groupby(['year','month']).agg({'value': ['mean']})['value']).opts(ylabel="Temperature", title='Year/Month Mean Temperature')
(count + mean).opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
year_maxmin = df.groupby(['year','month']).agg({'value': ['min', 'max']})
(hv.Bars(year_maxmin['value']['max']).opts(ylabel="Temperature", title='Year/Month Max Temperature') \
+ hv.Bars(year_maxmin['value']['min']).opts(ylabel="Temperature", title='Year/Month Min Temperature'))\
    .opts(opts.Bars(width=380, height=300,tools=['hover'],show_grid=True, stacked=True, legend_position='bottom'))

In [ ]:
hv.Distribution(df['value']).opts(opts.Distribution(title="Temperature Distribution", xlabel="Temperature", ylabel="Density", width=700, height=300,tools=['hover'],show_grid=True))

In [ ]:
((hv.Distribution(df.loc[df['year']==2013,'value'], label='2013') * hv.Distribution(df.loc[df['year']==2014,'value'], label='2014')).opts(title="Temperature by Year Distribution", legend_position='bottom') + \
(hv.Distribution(df.loc[df['month']==12,'value'], label='12') * hv.Distribution(df.loc[df['month']==1,'value'], label='1') \
     * hv.Distribution(df.loc[df['month']==2,'value'], label='2')).opts(title="Temperature by Month Distribution", legend_position='bottom')) \
     .opts(opts.Distribution(xlabel="Temperature", ylabel="Density", width=380, height=300,tools=['hover'],show_grid=True))

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Time Series Analysis

>plot temperature & its given anomaly points.

In [ ]:
anomalies = [[ind, value] for ind, value in zip(df[df['anomaly']==1].index, df.loc[df['anomaly']==1,'value'])]
(hv.Curve(df['value'], label="Temperature") * hv.Points(anomalies, label="Anomaly Points").opts(color='red', legend_position='bottom', size=2, title="Temperature & Given Anomaly Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
hv.Curve(df['value'].resample('D').mean()).opts(opts.Curve(title="Temperature Mean by Day", xlabel="Time", ylabel="Temperature", width=700, height=300,tools=['hover'],show_grid=True))

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 6. Modeling
>We will build several anomaly detection models and compare them each other.

## Model1. Hotelling's T2
><div class="alert alert-info" role="alert">
><ul>
><li>Basic anomaly detection method based on statustics.</li>
></ul>
></div>

In [ ]:
hotelling_df = pd.DataFrame()
hotelling_df['value'] = df['value']
mean = hotelling_df['value'].mean()
std = hotelling_df['value'].std()
hotelling_df['anomaly_score'] = [((x - mean)/std) ** 2 for x in hotelling_df['value']]
hotelling_df['anomaly_threshold'] = stats.chi2.ppf(q=0.95, df=1)
hotelling_df['anomaly']  = hotelling_df.apply(lambda x : 1 if x['anomaly_score'] > x['anomaly_threshold'] else 0, axis=1)

In [ ]:
(hv.Curve(hotelling_df['anomaly_score'], label='Anomaly Score') * hv.Curve(hotelling_df['anomaly_threshold'], label='Threshold').opts(color='red', line_dash="dotdash")) \
  .opts(title="Hotelling's T2 - Anomaly Score & Threshold", xlabel="Time", ylabel="Anomaly Score", legend_position='bottom').opts(opts.Curve(width=700, height=400, show_grid=True, tools=['hover']))

In [ ]:
anomalies = [[ind, value] for ind, value in zip(hotelling_df[hotelling_df['anomaly']==1].index, hotelling_df.loc[hotelling_df['anomaly']==1,'value'])]
(hv.Curve(hotelling_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="Hotelling's T2 - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
hotelling_f1 = f1_score(df['anomaly'], hotelling_df['anomaly'])
print(f'Hotelling\'s T2 F1 Score : {hotelling_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model2. One-Class SVM
><div class="alert alert-info" role="alert">
><ul>
><li>Unsupervised kernel-based anomaly detection method.</li>
></ul>
></div>

In [ ]:
ocsvm_model = OneClassSVM(nu=0.2, gamma=0.001, kernel='rbf')
ocsvm_ret = ocsvm_model.fit_predict(df['value'].values.reshape(-1, 1))
ocsvm_df = pd.DataFrame()
ocsvm_df['value'] = df['value']
ocsvm_df['anomaly']  = [1 if i==-1 else 0 for i in ocsvm_ret]

In [ ]:
anomalies = [[ind, value] for ind, value in zip(ocsvm_df[ocsvm_df['anomaly']==1].index, ocsvm_df.loc[ocsvm_df['anomaly']==1,'value'])]
(hv.Curve(ocsvm_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="One-Class SVM - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
ocsvm_f1 = f1_score(df['anomaly'], ocsvm_df['anomaly'])
print(f'One-Class SVM F1 Score : {ocsvm_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model3. Isolation Forest
><div class="alert alert-info" role="alert">
><ul>
><li>Unsupervised tree-based anomaly detection method.</li>
></ul>
></div>

In [ ]:
iforest_model = IsolationForest(n_estimators=300, contamination=0.1, max_samples=700)
iforest_ret = iforest_model.fit_predict(df['value'].values.reshape(-1, 1))
iforest_df = pd.DataFrame()
iforest_df['value'] = df['value']
iforest_df['anomaly']  = [1 if i==-1 else 0 for i in iforest_ret]

In [ ]:
anomalies = [[ind, value] for ind, value in zip(iforest_df[iforest_df['anomaly']==1].index, iforest_df.loc[iforest_df['anomaly']==1,'value'])]
(hv.Curve(iforest_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="Isolation Forest - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
sample_train = df['value'].values[np.random.randint(0, len(df['value']), (100))].reshape(-1, 1)
explainer = shap.TreeExplainer(model=iforest_model, feature_perturbation="interventional", data=sample_train)
shap_values = explainer.shap_values(X=sample_train)
shap.summary_plot(shap_values=shap_values, features=sample_train, feature_names=['value'], plot_type="violin")

In [ ]:
iforest_f1 = f1_score(df['anomaly'], iforest_df['anomaly'])
print(f'Isolation Forest F1 Score : {iforest_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model4. LOF
><div class="alert alert-info" role="alert">
><ul>
><li>Unsupervised density-based anomaly detection method, which measures the local deviation of density of a given sample with respect to its neighbors.</li>
></ul>
></div>

In [ ]:
lof_model = LocalOutlierFactor(n_neighbors=500, contamination=0.07)
lof_ret = lof_model.fit_predict(df['value'].values.reshape(-1, 1))
lof_df = pd.DataFrame()
lof_df['value'] = df['value']
lof_df['anomaly']  = [1 if i==-1 else 0 for i in lof_ret]

In [ ]:
anomalies = [[ind, value] for ind, value in zip(lof_df[lof_df['anomaly']==1].index, lof_df.loc[lof_df['anomaly']==1,'value'])]
(hv.Curve(lof_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="LOF - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
lof_f1 = f1_score(df['anomaly'], lof_df['anomaly'])
print(f'LOF F1 Score : {lof_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model5. ChangeFinder
><div class="alert alert-info" role="alert">
><ul>
><li>Change detection method based on SDAR model.</li>
></ul>
></div>

In [ ]:
cf_model = changefinder.ChangeFinder(r=0.002, order=1, smooth=250)
ch_df = pd.DataFrame()
ch_df['value'] = df['value']
ch_df['anomaly_score'] = [cf_model.update(i) for i in ch_df['value']]
ch_score_q1 = stats.scoreatpercentile(ch_df['anomaly_score'], 25) 
ch_score_q3 = stats.scoreatpercentile(ch_df['anomaly_score'], 75) 
ch_df['anomaly_threshold'] = ch_score_q3 + (ch_score_q3 - ch_score_q1) * 3
ch_df['anomaly']  = ch_df.apply(lambda x : 1 if x['anomaly_score'] > x['anomaly_threshold'] else 0, axis=1)

In [ ]:
(hv.Curve(ch_df['anomaly_score'], label='Anomaly Score') * hv.Curve(ch_df['anomaly_threshold'], label='Threshold').opts(color='red', line_dash="dotdash")) \
  .opts(title="ChangeFinder - Anomaly Score & Threshold", xlabel="Time", ylabel="Anomaly Score", legend_position='bottom').opts(opts.Curve(width=700, height=400, show_grid=True, tools=['hover']))

In [ ]:
anomalies = [[ind, value] for ind, value in zip(ch_df[ch_df['anomaly']==1].index, ch_df.loc[ch_df['anomaly']==1,'value'])]
(hv.Curve(ch_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="ChangeFinder - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
ch_f1 = f1_score(df['anomaly'], ch_df['anomaly'])
print(f'ChangeFinder F1 Score : {ch_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model6. Variance Based Method
><div class="alert alert-info" role="alert">
><ul>
><li>This is variance based method with assumption of the normal distribution against the data.</li>
></ul>
></div>

In [ ]:
sigma_df = pd.DataFrame()
sigma_df['value'] = df['value']
std = sigma_df['value'].std()
sigma_df['anomaly_threshold_3r'] = mean + 1.5*std
sigma_df['anomaly_threshold_3l'] = mean - 1.5*std
sigma_df['anomaly']  = sigma_df.apply(lambda x : 1 if (x['value'] > x['anomaly_threshold_3r']) or (x['value'] < x['anomaly_threshold_3l']) else 0, axis=1)

In [ ]:
anomalies = [[ind, value] for ind, value in zip(sigma_df[sigma_df['anomaly']==1].index, sigma_df.loc[sigma_df['anomaly']==1,'value'])]
(hv.Curve(sigma_df['value'], label="Temperature") * hv.Points(anomalies, label="Detected Points").opts(color='red', legend_position='bottom', size=2, title="Variance Based Method - Detected Points"))\
    .opts(opts.Curve(xlabel="Time", ylabel="Temperature", width=700, height=400,tools=['hover'],show_grid=True))

In [ ]:
sigma_f1 = f1_score(df['anomaly'], sigma_df['anomaly'])
print(f'Variance Based Method F1 Score : {sigma_f1}')

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

## Model Comparison

In [ ]:
display(HTML('<h3>Evaluation - F1 Score</h3>'+tabulate([['F1 Score', hotelling_f1, ocsvm_f1, iforest_f1, lof_f1, ch_f1, sigma_f1]],\
                      ["", "Hotelling's T2", "One-Class SVM", 'Isolation Forest', 'LOF', 'ChangeFinder', 'Variance Based Method'], tablefmt="html")))

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 7. Conclusion
><div class="alert alert-success" role="alert">
><ul>
><li>The model built with <b>simple algorithms</b> resulted in high accuracy.</li>
><li>It is considered that a simple algorithm is effective in many cases <u>for time-series data having a simple structure</u> such as a constant mean and variance over time.</li>
><li>Complex and robust algorithms are considered to be effective <u>when time-series has complicated patterns or when there are various anomolous patterns</u>.</li>
></ul>
>In this analysis, the model was evaluated based on the F1 score, but if you want to make <b>a more business-oriented evaluation</b>, the following viewpoints should be added to the evaluation.
><ul>
><li>Detection of signs of anomalies</li>
><li>Balance of importance of false positive and false negative</li>
><li>Model interpretability</li>
></ul>
></div>

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>

# 8. References
> * **NAB Anomaly Points References**  
> https://github.com/numenta/NAB/blob/master/labels/combined_windows.json  
> * **Anomaly Detection Learning Resources**  
> https://github.com/yzhao062/anomaly-detection-resources  
> * **PyOD documentation**  
> https://pyod.readthedocs.io/en/latest/  
> * **Anomaly Detection Toolkit documentation**  
> https://arundo-adtk.readthedocs-hosted.com/en/latest/  

<a href="#top" class="btn btn-success btn-sm active" role="button" aria-pressed="true" style="color:white;">Table of Contents</a>